In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:
data_root = 'C:/Users/Luis/Documents/Ciencia de Datos/DatasetDeeplearnig/'

train_folders = os.listdir(data_root+'notMNIST_large/')
test_folders =  os.listdir(data_root+'notMNIST_small/')

print(train_folders)


# Problem 1 
Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

In [ ]:
def muestra_imagen(nombreImagen):
    display(Image(filename=nombreImagen))

def ta_muestra(carpetas, tmuestra):
    tmuestra_ini = int(tmuestra/len(carpetas)) # distribucion uniforme de muestras en carpetas
    estratos = len(carpetas)
    muestra = tmuestra-((estratos-1)*tmuestra_ini)
    return muestra
    

def list_imagens(carpetas, tmuestra):
    num_image_file = []

    for carpeta in carpetas:
        cant_imgs = sum([len(files) for r, d, files in os.walk(data_root+'notMNIST_large/'+carpeta+'/')])
        num_image_file.append({'nombre': data_root+'notMNIST_large/'+carpeta+'/',
                               'cantidad': cant_imgs, 'muestra': ta_muestra(carpetas, tmuestra)})
        
        print(data_root+'notMNIST_large/'+carpeta+'/')
        print(cant_imgs)

    return num_image_file




In [ ]:
sample_train = 30
sample_test = 15


lista_de_todas_imgs_train = list_imagens(train_folders, sample_train)


lista_de_todas_imgs_test  = list_imagens(test_folders, sample_test)

In [ ]:
# imprime muestra de imagenes de entrenamiento

for carpeta in lista_de_todas_imgs_train:
    muestras = np.sort(np.random.randint(0, carpeta['cantidad'], size= carpeta['muestra']))

    arch_imagen = [image_files for r, d, image_files in os.walk(carpeta['nombre'])]
    arch_imagen = arch_imagen[0]
    
    print(carpeta['nombre']+'\n')
    
    for muestra in muestras:
#        print(arch_imagen[muestra])
        muestra_imagen(carpeta['nombre']+arch_imagen[muestra])
       
print('xd')

            


Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road.

A few images might not be readable, we'll just skip them.

In [ ]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (imageio.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except (IOError, ValueError) as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(raiz, data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = raiz + folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(raiz+folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(data_root+'notMNIST_large/', train_folders, 45000)
test_datasets = maybe_pickle(data_root+'notMNIST_small/', test_folders, 1800)

# Problem 2
Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

In [ ]:
def maybe_unpickle(raiz, data_folder):
    set_filename = raiz + data_folder
    dataset = None

    if os.path.exists(set_filename):
        print('Deserializando %s.' % set_filename)
        try:
            with open(set_filename, 'rb') as f:
                dataset = pickle.load(f, fix_imports=True, encoding="ASCII", errors="strict")
        except Exception as e:
            print('Unable to load data to', set_filename, ':', e)
    else:
        print('%s unable file - Skipping unpickling.' % set_filename) 
    return dataset

def muestra_letras(root, dicc, n):
    for label, file in dicc.items():
        datos = maybe_unpickle(root, file)
        est_muestra = np.sort(np.random.randint(0, len(datos), size= n))
        print(label)
        
        for estrac_muestra in est_muestra:
            ver_letra(datos[estrac_muestra])
    return

def ver_letra(datos, titulo = '', guarda = False, figsize = .70):
    imgplot = plt.figure(figsize=[figsize, figsize] ) # equivalencia entre pulgadas y pixeles
    imgplot = plt.axis('off')
    imgplot = plt.gray()
    imgplot = plt.title(titulo)
    imgplot = plt.imshow(datos)

    if guarda: 
        imgplot = plt.savefig(data_root+'letra.png')

    plt.show()
    
    return

def label_real(num):
    dic = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I', 9:'J'}
    letra = dic[num]
    return letra


In [ ]:
train_dic = {train_folders[i]: train_folders[i+1] for i in range(0, len(train_folders), 2)}

test_dic = {test_folders[i]: test_folders[i+1] for i in range(0, len(test_folders), 2)}

tam_muestra = 3 # tres muestras por cada letra

root_train = data_root+'notMNIST_large/'
root_test  = data_root+'notMNIST_small/'

In [ ]:
muestra_letras(root_train, train_dic, tam_muestra)

# muestra_letras(root_test, test_dic, tam_muestra)

# Problem 3
Another check: we expect the data to be balanced across classes. Verify that.

In [ ]:
# para hacer check vamos a recorrer ambos diccionarios (test y train), para modificar
# el valor agregando ademas de archivo de almacenamiento, la cantidad de unidades de 
# datos que contiene

def cant_letras(root, dicc):
    cantidad = []
    for label, file in dicc.items():
        datos = maybe_unpickle(root, file)
        print('-')
        cantidad.append(len(datos))
    return cantidad

def dic_builder(root, folder_pri, dic):
    label = [letra for letra in folder_pri if len(letra)==1]
    cant = cant_letras(root, dic)
    data = dict(zip(label, cant))
    return data

def plt_bar(dat, title):
    plt.bar(range(len(dat)), list(dat.values()), align='center')
    plt.xticks(range(len(dat)), list(dat.keys()))
    plt.title(title)
    plt.show()

In [ ]:
train_dat = dic_builder(root_train, train_folders, train_dic)
test_dat = dic_builder(root_test, test_folders, test_dic)

tits = ['Grupo de Entrenamiento', 'Grupo de Prueba']
dats = [train_dat, test_dat]

data = dict(zip(tits, dats))

for tit, dat in data.items():
    plt_bar(dat, tit)

# data to be balanced across classes

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune train_size as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [ ]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels

In [ ]:
# train_dic y test_dic se obtienen en resolucion de problema 2 (muestra de imagenes)
# image_size se obtiene en probl 1

image_size = 28  # Pixel width and height.
train_datasets = [root_train+nombre for nombre in list(train_dic.values())]
test_datasets  = [root_test+nombre for nombre in list(test_dic.values())]

train_size = 50000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)


Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [ ]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

# Next, we'll randomize the data. 
# It's important to have the labels well shuffled for the training
# and test distributions to match
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)



# Problem 4
Convince yourself that the data is still good after shuffling!

In [ ]:
# tamaño de muestra = 30 je je

t_muestra = 30 # para cada conjunto de datos (label)
ndatos = [train_labels.shape, valid_labels.shape, test_labels.shape]
muestras = [np.sort(np.random.randint(0, ndato[0]-1, size= t_muestra)) for ndato in ndatos]

pob_dat = ['train_dataset', 'valid_dataset', 'test_dataset']
pob_lab = ['train_labels', 'valid_labels', 'test_labels']

data  = dict(zip(pob_dat, muestras))
label = dict(zip(pob_lab, muestras))

# extrayendo las muestras y ..
for tit, dat in data.items():
    pobl = vars()[tit]
    data[tit] = pobl[dat]
# sus respectivas labels,
for tit, dat in label.items():
    pobl = vars()[tit]
    label[tit] = pobl[dat] #~.

data  = { k.replace(k, k[0:k.find('_')]): v for k, v in data.items()  }
label = { k.replace(k, k[0:k.find('_')]): v for k, v in label.items() }



In [ ]:
from itertools import chain
from collections import defaultdict

label_data = defaultdict(list)
for k, v in chain(label.items(), data.items()):
    label_data[k].append(v)
 
#print(label_data.keys())
#print(label_data['train'][0])
#print(label_data['train'][0][0])
#print(len(label_data['train'][1][0]))

for clave, valor in label_data.items():
    print("-/-/-/-/-/-/-/-/-/-/-/-/-/")
    print('Conjunto: ', clave)

    for i in range(len(valor[0])):
        ver_letra(valor[1][i], label_real(valor[0][i]))

Finally, let's save the data for later reuse:

In [ ]:
pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [ ]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

# Problem 5
By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it. Measure how much overlap there is between training, validation and test samples.

Optional questions:

What about near duplicates between datasets? (images that are almost identical)
Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.

In [ ]:
# leer archivo guardado previamente
dataset = maybe_unpickle(data_root, "notMNIST.pickle")

for k, v in dataset.items():
    print(k, 'cant: ', len(v))



In [ ]:
# para ello se usara una funcion hash para cada imagen

import imagehash
from PIL import Image

ver_letra(dataset['train_dataset'][0], label_real(dataset['train_labels'][0]), True)
hash0  = imagehash.average_hash(Image.open(data_root+'letra.png'))
hash01 = imagehash.dhash(Image.open(data_root+'letra.png'))
print('hash: ', hash01)

letra2 = ver_letra(dataset['train_dataset'][1], label_real(dataset['train_labels'][1]), True)
hash1  = imagehash.average_hash(Image.open(data_root+'letra.png'))
hash02 = imagehash.dhash(Image.open(data_root+'letra.png'))
print('hash: ', hash02)

i = 0
for letra in dataset['train_dataset']:
    i += 1
    ver_letra(letra, '', True)

    if i>10:
        break
    


In [ ]:
print(hash01==hash02)
print((hash01 - hash02)/len(hash01.hash)**2)